## 季節など周期性で売り上げ予測(時系列分析)

目的
時系列予測モデルを使った売り上げ予測の方法と

時系列予測は、回帰と同様に値を予測するものです。

*   目的変数(予測する値)の過去データを使って予測すること
*   「信頼区間」という　ここからここまでの間に80％の確立で値が存在するということを示せること

が異なる特徴となります。



### 共通事前準備

In [1]:
# 日本語化ライブラリ導入
!pip install japanize-matplotlib | tail -n 1

In [2]:
# 共通事前処理

# 余分なワーニングを非表示にする
import warnings
warnings.filterwarnings('ignore')

# 必要ライブラリのimport
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# matplotlib日本語化対応
import japanize_matplotlib

# データフレーム表示用関数
from IPython.display import display

# 表示オプション調整
# numpyの浮動小数点の表示精度
np.set_printoptions(suppress=True, precision=4)

# pandasでの浮動小数点の表示精度
pd.options.display.float_format = '{:.4f}'.format

# データフレームですべての項目を表示
pd.set_option("display.max_columns",None)

# グラフのデフォルトフォント指定
plt.rcParams["font.size"] = 14

# 乱数の種
random_seed = 123

オリジナルURL   
https://archive.ics.uci.edu/ml/datasets/bike+sharing+dataset

#### データ項目メモ

instant インデックス  
dteday 日付(yy-mm-dd)  
season 季節 (1: 冬 2: 春 3: 夏 4:秋)  
yr 年 (0: 2011, 1:2012)  
mnth 月  (1 - 12)  
hr 時間  (0 - 23)  
holiday 祝日  
weekday 曜日 (0 - 6)  
workingday  勤務日 (1: 勤務日 0: 休日)  
weathersit 天気 (1: 晴れから曇り 2: 霧 3: 小雨 4: 大雨)  
temp 気温 (正規化済み)  
atemp 体感気温 (正規化済み)  
hum 湿度 (正規化済み)  
windspeed 風速 (正規化済み)  
casual 臨時ユーザー利用数  
registered 登録ユーザー利用数  
cnt 全体ユーザー利用数  

### データ読み込みからデータ確認まで

#### データ読み込み

データは、回帰の予測の時に利用した

バイクシェアリングの利用者数の過去データに天気や祝日などの情報を加えたデータをもとに予測を行ってみます。



In [ ]:
# ダウンロード元URL
url = 'https://archive.ics.uci.edu/ml/\
machine-learning-databases/00275/\
Bike-Sharing-Dataset.zip'

# 公開データのダウンロードと解凍
!wget $url -O Bike-Sharing-Dataset.zip | tail -n 1
!unzip -o Bike-Sharing-Dataset.zip | tail -n 1

In [4]:
# day.csvをデータフレームに取り込み
# 日付を表す列はparse_datesで指定する
df = pd.read_csv('day.csv', parse_dates=[1])

In [5]:
# instant は連番で予測で不要なので削除
df = df.drop('instant', axis=1)

# 項目名の日本語化

columns = [
    '日付',  '季節',  '年', '月', '祝日', '曜日', '勤務日', '天気', 
    '気温', '体感温度',  '湿度', '風速',
    '臨時ユーザー利用数', '登録ユーザー利用数', '全体ユーザー利用数'
]

# 項目名を日本語に置き換え
df.columns = columns

#### データ確認

In [ ]:
# 先頭5行の確認
display(df.head())

# 最終5行の確認
# display(df.tail())

### データ前処理とデータ分割

#### データ前処理

目的変数と、日付だけのデータフレームを作成する
※アルゴリズムの制限で列名を変更する

In [ ]:
# 「日付」と「登録ユーザー利用数」のみ抽出し、
# 列名を日付:ds 、登録ユーザー利用数:y に置き換えたデータフレームdf2を作る

# データフレーム全体のコピー
df2 = df.copy()

# 「日付」「登録ユーザー利用数」列の抽出
df2 = df2[['日付', '登録ユーザー利用数']]

# 列名の置き換え
df2.columns = ['ds', 'y']

# 結果確認
display(df2.head())

#### データ分割
2012年11月1日より前を訓練データ(x_train)に、後ろを検証データ(x_test)にする

In [8]:
# 分割日 mdayの設定
mday = pd.to_datetime('2012-11-1')

# 訓練用indexと検証用indexを作る
train_index = df2['ds'] < mday
test_index = df2['ds'] >= mday

# 入力データの分割
x_train = df2[train_index]
x_test = df2[test_index]

# 日付データの分割(グラフ表示用)
dates_test = df2['ds'][test_index]

### アルゴリズム選定

まずは、FaceBookが開発したProphet　という　モデルで予測を行っていきます。
その他のアルゴリズムについては、この後に、ライブラリ群とともに紹介します。

\\

**Prophet**

*   「日単位」、「週単位」、「月単位」などの周期関数とデータ期間でのトレンドを示すことができます。

*   時系列予測のデータは、日時と値の組み合わせとシンプルでしたが、天気や祝日などの回帰モデルで利用した情報も含めて時系列予測ができます。


*   予測結果が出るまでの時間も早く、比較的簡単に利用できます。   

*   導入事例も比較的多いです。   



    growth:傾向変動の関数．既定値は線形'linear'．ロジスティック曲線にするに'logistic'に設定する．
    changepoints : 傾向変更点のリスト
    changepoint_range : 傾向変化点の候補の幅（先頭から何割を候補とするか）
    n_changepoints : 傾向変更点の数
    yearly_seasonality='auto' : 年次の季節変動を考慮するか否か
    weekly_seasonality='auto' : 週次の季節変動を考慮するか否か
    daily_seasonality='auto' : 日時の季節変動を考慮するか否か
    holidays : 休日のリスト
    seasonality_prior_scale : 季節変動の事前分布のスケール値（パラメータの柔軟性をあら表す）
    holidays_prior_scale : 休日のの事前分布のスケール値（パラメータの柔軟性をあら表す）
    changepoint_prior_scale : 傾向変更点の事前分布のスケール値（パラメータの柔軟性をあら表す）
    mcmc_samples=0 : MCMC法のサンプル数
    interval_width=0.80 : 不確実性の幅
    uncertainty_samples=1000 : 不確実性の幅を計算する際のサンプル数



## モデル選定
3つのseasonalityパラメータの設定が重要 

今回のデータの場合、日単位のデータなのでdaily_seasonalityは不要

weekly_seasonality とdaily_seasonalityは 

True / Falseの他に数値で指定することも可能 (三角関数の個数)
seasonality_mode: additive(デフォルト) multiplicative

In [10]:
# ライブラリのimport
from fbprophet import Prophet

m1 = Prophet(yearly_seasonality=True, weekly_seasonality=True, 
    daily_seasonality=False,
    seasonality_mode='multiplicative')

### 学習・予測 　目的変数の過去データのみ場合

In [ ]:
# 学習
m1.fit(x_train)

In [ ]:
# 予測用データの作成
# (日付 ds だけの入ったデータフレーム)
# 61は予測したい日数 (2012-11-1 から2012-12-31)
future1 = m1.make_future_dataframe(periods=61, freq='D')

# 結果確認
display(future1.head())
display(future1.tail())

In [13]:
# 予測
# 結果はデータフレームで戻ってくる
fcst1 = m1.predict(future1)

### 評価　目的変数の過去データのみ場合

In [ ]:
# 要素ごとのグラフ描画
# この段階ではトレンド、週周期、年周期
fig = m1.plot_components(fcst1)
plt.show()

#### 訓練データ・検証データ全体のグラフ化　目的変数の過去データのみ場合

In [ ]:
# 訓練データ・検証データ全体のグラフ化
fig, ax = plt.subplots(figsize=(10,6))

# 予測結果のグラフ表示(prophetの関数)
m1.plot(fcst1, ax=ax)

# タイトル設定など
ax.set_title('登録ユーザー利用数予測')
ax.set_xlabel('日付')
ax.set_ylabel('利用数')

# グラフ表示
plt.show()

#### R2値の計算

予測結果をScoreにして確認する
\
回帰と同様にR2値を求める

In [16]:
# ypred1: fcst1から予測部分のみ抽出する
ypred1 = fcst1[-61:][['yhat']].values

# ytest1: 予測期間中の正解データ
ytest1 = x_test['y'].values

# R2値の計算
from sklearn.metrics import r2_score
score = r2_score(ytest1, ypred1)

# 結果確認
print(f'R2 score:{score:.4f}')

R2 score:0.3725


#### 予測期間中のグラフ表示(正解データと予測結果)

In [ ]:
# 時系列グラフの描画 
import matplotlib.dates as mdates
fig, ax = plt.subplots(figsize=(8, 4))

# グラフ描画
ax.plot(dates_test, ytest1, label='正解データ', c='k')
ax.plot(dates_test, ypred1, label='予測結果', c='b')

# 日付目盛間隔
# 木曜日ごとに日付を表示
weeks = mdates.WeekdayLocator(byweekday=mdates.TH)
ax.xaxis.set_major_locator(weeks)

# 日付表記を90度回転
ax.tick_params(axis='x', rotation=90)

# 方眼表示など
ax.grid()
ax.legend()
ax.set_title('登録ユーザー利用数予測結果')

# 画面出力
plt.show()

## チューニング方針

* 1 「休日」を特別な日として追加  
* 2 回帰モデルに「天気」「気温」「風速」「湿度」を追加


### チューニング1 休日を特別な日としてデータに追加

#### データ前処理

目的変数と、日付のデータフレームに加え祝日をとくべつな日としてデータを加えて\
データフレームを作成する

lowere_window/uppe_windowは、イベントが前後何日に影響するかを指定できる値です。


祝日なので、影響を考慮するのは難しいかもしれませんが、土日との連休などは影響を\
考慮するとよい予測結果が出るかもしれません。


祝日の日付だけを抽出します。その日付に対して、情報を追加します。


※アルゴリズムの制限で列名を変更する

In [ ]:
# 休日の抽出
df_holiday = df[df['祝日']==1]
holidays = df_holiday['日付'].values

# データフレーム形式に変換
df_add = pd.DataFrame({'holiday': 'holi',
    'ds': holidays,
    'lower_window': 0,
    'upper_window': 0
})

# 結果確認
display(df_add.head())     
display(df_add.tail())                

### 学習・予測 　休日を特別な日としてデータに追加

In [19]:
# 休日(df_add)をモデルの入力とする

# アルゴリズム選定
# holidaysパラメータを追加してモデルm2を生成
m2 = Prophet(yearly_seasonality=True, 
    weekly_seasonality=True, daily_seasonality=False,
    holidays = df_add, seasonality_mode='multiplicative')

# 学習
m2 = m2.fit(x_train)

# 予測
fcst2 = m2.predict(future1)

### 評価　休日を特別な日としてデータに追加

In [ ]:
# 要素ごとのグラフ描画
fig = m2.plot_components(fcst2)
plt.show()

In [ ]:
# 訓練データ・検証データ全体のグラフ化
fig, ax = plt.subplots(figsize=(10,6))

# 予測結果のグラフ表示(prophetの関数)
m2.plot(fcst2, ax=ax)

# タイトル設定など
ax.set_title('登録ユーザー利用数予測')
ax.set_xlabel('日付')
ax.set_ylabel('利用数')

# グラフ表示
plt.show()

R2値の計算

予測結果をScoreにして確認する
回帰と同様にR2値を求める

In [ ]:
# R値の計算

# fcst2から予測部分のみ抽出する
ypred2 = fcst2[-61:][['yhat']].values

# R2値の計算
score2 = r2_score(ytest1, ypred2)

# 結果確認
r2_text2 = f'R2 score:{score2:.4f}'
print(r2_text2)

#### 予測期間中のグラフ表示(正解データと予測結果)

In [ ]:
# 時系列グラフの描画 
import matplotlib.dates as mdates
fig, ax = plt.subplots(figsize=(8, 4))

# グラフ描画
ax.plot(dates_test, ytest1, label='正解データ', c='k')
ax.plot(dates_test, ypred1, label='予測結果v1', c='c')
ax.plot(dates_test, ypred2, label='予測結果v2', c='b')

# 日付目盛間隔
# 木曜日ごとに日付を表示
weeks = mdates.WeekdayLocator(byweekday=mdates.TH)
ax.xaxis.set_major_locator(weeks)

# 日付表記を90度回転
ax.tick_params(axis='x', rotation=90)

# 開始日と終了日
sday = pd.to_datetime('2012-11-1')
eday = pd.to_datetime('2013-1-1')
ax.set_xlim(sday, eday) 

# 方眼表示など
ax.grid()
ax.legend()
ax.set_title('登録ユーザー利用数予測結果  ' + r2_text2)

# 画面出力
plt.show()

### チューニング2 「天気」「気温」「風速」「湿度」をデータに追加


#### データ前処理

目的変数と、祝日のデータと「天気」「気温」「風速」「湿度」

lowere_window/uppe_windowは、イベントが前後何日に影響するかを指定できる値です。


祝日なので、影響を考慮するのは難しいかもしれませんが、土日との連休などは影響を\
考慮するとよい予測結果が出るかもしれません。


祝日の日付だけを抽出します。その日付に対して、情報を追加します。


祝日データの入れ方とそれ以外の情報をモデルに入れる時の入れ方が違うので\
チューニングを2つに分けて紹介しています。

※アルゴリズムの制限で列名を変更する

In [ ]:
# 学習データに「天気」「気温」「風速」「湿度」を追加
df3 = pd.concat([df2, df[['天気', '気温', '風速', '湿度']]], axis=1)

# 入力データの分割
x2_train = df3[train_index]
x2_test = df3[test_index]

# 結果確認
display(x2_train.tail())

In [ ]:
# アルゴリズム選定

m3 = Prophet(yearly_seasonality=True, 
    weekly_seasonality=True, daily_seasonality=False,
    seasonality_mode='multiplicative', holidays = df_add)

#  add_regressor関数で、「天気」「気温」「風速」「湿度」をモデルに組み込む
m3.add_regressor('天気')
m3.add_regressor('気温')
m3.add_regressor('風速')
m3.add_regressor('湿度')

# 学習
m3.fit(x2_train)

In [27]:
# 予測用の入力データを作る
future3 = df3[['ds', '天気', '気温', '風速', '湿度']]

# 予測
fcst3 = m3.predict(future3)

### ステップ2の評価

In [ ]:
# 要素ごとのグラフ描画
fig = m3.plot_components(fcst3)
plt.show()

In [ ]:
# R値の計算

# fcstから予測部分のみ抽出する
ypred3 = fcst3[-61:][['yhat']].values
score3 = r2_score(ytest1, ypred3)

# 結果確認
r2_text3 = f'R2 score:{score3:.4f}'
print(r2_text3)

In [ ]:
# 時系列グラフの描画 
import matplotlib.dates as mdates
fig, ax = plt.subplots(figsize=(8, 4))

# グラフ描画
ax.plot(dates_test, ytest1, label='正解データ', c='k')
ax.plot(dates_test, ypred2, label='予測結果v2', c='c')
ax.plot(dates_test, ypred3, label='予測結果v3', c='b')

# 日付目盛間隔
# 木曜日ごとに日付を表示
weeks = mdates.WeekdayLocator(byweekday=mdates.TH)
ax.xaxis.set_major_locator(weeks)

# 日付表記を90度回転
ax.tick_params(axis='x', rotation=90)

# 方眼表示など
ax.grid()
ax.legend()
ax.set_title('登録ユーザー利用数予測結果  ' + r2_text3)

# 画面出力
plt.show()

In [ ]:
# 訓練データ・検証データ全体のグラフ化
fig, ax = plt.subplots(figsize=(10,6))

# 予測結果のグラフ表示(prophetの関数)
m3.plot(fcst3, ax=ax)

# タイトル設定など
ax.set_title('登録ユーザー利用数予測')
ax.set_xlabel('日付')
ax.set_ylabel('利用数')

# グラフ表示
plt.show()